In [44]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import shutil
import hashlib
from collections import defaultdict
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

eficientnet -> transformers -> camadas -> fine tuning -> svm

In [ ]:
df_train = pd.read_csv('archive/train.csv')


In [ ]:
sampled_df = df_train.groupby('label', group_keys=False).apply(lambda x: x.sample(min(len(x), 1000)))


In [ ]:
X = sampled_df['file_name']
Y = sampled_df['label']
Y

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
X_train

In [ ]:
caminho_pasta_nova_ia = 'images/IA/'
caminho_pasta_nova_human = 'images/HUMAN/'
caminho_pasta_imagens= 'archive/train_data/'
nome_imagens_df = X_train.values
caminhodo = '/home/leonardo/Downloads/testando_cv/'
caminho_teste_ia = 'validation/IA/'
caminho_teste_human = 'validation/HUMAN/'

In [ ]:
for arquivo in os.listdir(caminho_pasta_imagens):
    if ';' in arquivo:  # Confere se o arquivo tem o padrão ;1 ou ;0
        nome_imagem = arquivo.split(';')[0]  # Extrai o nome da imagem sem o ;1 ou ;0
        
        caminho_antigo = os.path.join(caminho_pasta_imagens, arquivo)
        caminho_novo = os.path.join(caminho_pasta_imagens, nome_imagem)
        
        # Renomeia o arquivo
        os.rename(caminho_antigo, caminho_novo)

In [ ]:


# Loop sobre cada nome de imagem no DataFrame
for nome_imagem, label in zip(X_train.values, Y_train. values):  # Supondo que a primeira coluna seja o nome das imagens
    nome_imagem = nome_imagem[11:]
    caminho_imagem_origem = os.path.join(caminho_pasta_imagens, nome_imagem)
    caminho_imagem_origem = os.path.join(caminhodo, caminho_imagem_origem)
    print(caminho_imagem_origem)
    if label == 1:
        caminho_imagem_destino = os.path.join(caminho_pasta_nova_ia, nome_imagem)
    if label == 0:
        caminho_imagem_destino = os.path.join(caminho_pasta_nova_human, nome_imagem)

    
    # Mover a imagem se existir
    if os.path.exists(caminho_imagem_origem):
        shutil.move(caminho_imagem_origem, caminho_imagem_destino)
    else:
        print('nao achei')
        
    

In [ ]:


def contar_arquivos(diretorio):
    return len([arquivo for arquivo in os.listdir(diretorio) if os.path.isfile(os.path.join(diretorio, arquivo))])

# Caminho do diretório que você quer contar os arquivos
caminho_diretorio = 'images/IA/'

total_arquivos = contar_arquivos(caminho_diretorio)
print(f"Total de arquivos no diretório: {total_arquivos}")


In [ ]:


def calcular_hash(arquivo, buffer_size=65536):
    sha256 = hashlib.sha256()
    with open(arquivo, 'rb') as f:
        while chunk := f.read(buffer_size):
            sha256.update(chunk)
    return sha256.hexdigest()

def encontrar_arquivos_iguais(diretorio):
    hashes = defaultdict(list)

    for raiz, _, arquivos in os.walk(diretorio):
        for arquivo in arquivos:
            caminho_arquivo = os.path.join(raiz, arquivo)
            hash_arquivo = calcular_hash(caminho_arquivo)
            hashes[hash_arquivo].append(caminho_arquivo)
    
    # Retornar apenas os arquivos que possuem cópias
    arquivos_duplicados = {hash_: caminhos for hash_, caminhos in hashes.items() if len(caminhos) > 1}
    
    return arquivos_duplicados

# Caminho do diretório que você quer analisar
caminho_diretorio = 'images/IA/'

duplicados = encontrar_arquivos_iguais(caminho_diretorio)

if duplicados:
    print("Arquivos duplicados encontrados:")
    for hash_, arquivos in duplicados.items():
        print(f"\nHash: {hash_}")
        for arquivo in arquivos:
            print(f" - {arquivo}")
else:
    print("Nenhum arquivo duplicado encontrado.")


In [ ]:
caminho_pasta_nova_ia = 'images/IA/'
caminho_pasta_nova_human = 'images/HUMAN/'
caminho_pasta_imagens= 'archive/train_data/'
nome_imagens_df = X_train.values
caminhodo = '/home/leonardo/Downloads/testando_cv/'
caminho_teste_ia = 'validation/IA/'
caminho_teste_human = 'validation/HUMAN/'

In [ ]:
Y_test

In [ ]:


# Loop sobre cada nome de imagem no DataFrame
for nome_imagem, label in zip(X_test.values, Y_test.values):  # Supondo que a primeira coluna seja o nome das imagens
    nome_imagem = nome_imagem[11:]
    caminho_imagem_origem = os.path.join(caminho_pasta_imagens, nome_imagem)
    caminho_imagem_origem = os.path.join(caminhodo, caminho_imagem_origem)
    print(caminho_imagem_origem)
    if label == 1:
        caminho_imagem_destino = os.path.join(caminho_teste_ia, nome_imagem)
    if label == 0:
        caminho_imagem_destino = os.path.join(caminho_teste_human, nome_imagem)

    
    # Mover a imagem se existir
    if os.path.exists(caminho_imagem_origem):
        shutil.move(caminho_imagem_origem, caminho_imagem_destino)
    else:
        print('nao achei')

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255)

treino_gen = datagen.flow_from_directory(
    'images/',  # Caminho para o diretório de treino
    target_size=(224, 224),  # Redimensiona todas as imagens para 224x224
    batch_size=32,           # Número de imagens carregadas por vez
    class_mode='binary'  # Tipo de classificação (categorical para múltiplas classes)
)

# Carregar imagens de teste
teste_gen = datagen.flow_from_directory(
    'validation/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 1624 images belonging to 2 classes.
Found 390 images belonging to 2 classes.


In [15]:
print(treino_gen.class_indices)  # {'classe1': 0, 'classe2': 1}


{'HUMAN': 0, 'IA': 1}


In [23]:
treino_gen

In [ ]:
model_cnn_basic_1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=10,
                           kernel_size = 3,
                           activation="relu",
                           input_shape=(224,224,3)),
    tf.keras.layers.Conv2D(10,3,activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2, padding="valid"),
    tf.keras.layers.Conv2D(10,3,activation="relu"),
    tf.keras.layers.Conv2D(10,3,activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2, padding="valid"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model_cnn_basic_1.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model_cnn_basic_1.fit(treino_gen, epochs=5, validation_data=teste_gen, steps_per_epoch=len(treino_gen))


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [35]:
from PIL import Image

try:
    img = Image.open("images/HUMAN/0a9fa1ee019a4781aa66049b7ed5475d.jpg")
    img.show()
    print("Pillow está funcionando corretamente.")
except Exception as e:
    print(f"Erro ao abrir a imagem: {e}")

Pillow está funcionando corretamente.


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/leonardo/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE


In [45]:
import PIL
print(PIL.__version__)  # Deve retornar '11.0.0'

11.1.0


In [51]:
print(treino_gen.samples)



1624
